In [95]:
import exif
from PIL import Image
import glob
import tiffile as js
import json
import numpy as np
from PIL.ExifTags import GPSTAGS
from PIL.ExifTags import TAGS
from pprint import pprint
from PIL import Image
import piexif
import json


codec = 'ISO-8859-1'  # or latin-1

#display exif data
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()

# Get metadata information

def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")
    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")
            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

# Get key,and value
def get_labeled_exif(exif):
    labeled = {}
    for (key, val) in exif.items():
        labeled[TAGS.get(key)] = val
    return labeled

#print(get_labeled_exif(filename))

#convert lat, alt to mapping coordinates
def get_decimal_from_dms(dms, ref):
#     degrees = dms[0]
#     minutes = dms[1] / 60.0
#     seconds = dms[2] / 3600.0
    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0  
    if ref in ['S', 'E']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)

#import files and check file types
def main():
    coor = []
    all_info = []
    root = glob.glob("./img/*.jpg")

    for filename in root:
            if filename.endswith(('jpg','JPG','png','PNG','tiff','TIFF')):
                img = filename[6:]
                exif = get_exif(filename)
                labeled = get_labeled_exif(exif)
                #pprint(labeled)
                geotags = get_geotagging(exif)
                geo = get_coordinates(geotags)
                coor.append([img, geo])
                all_info.append(labeled)
    #pprint(coor)
    pprint(all_info)
    lst = []
    for child in coor:
        info = ["img_name", "gps"]
        lst1 = {k: v for k, v in zip(info, child)}
        lst.append(lst1)
    pprint(lst)
    with open('data.json', 'w') as f:
        json.dump(lst, f)
#     with open('all_info.json', 'w') as file:
#         json.dump(all_info, file)



if __name__ == '__main__':
    main()

[{None: '+09:00',
  'ApertureValue': (214, 100),
  'ColorSpace': 1,
  'DateTime': '2021:06:03 20:11:43',
  'DateTimeDigitized': '2021:06:03 20:11:43',
  'DateTimeOriginal': '2021:06:03 20:11:43',
  'DigitalZoomRatio': (200, 100),
  'ExifImageHeight': 1908,
  'ExifImageWidth': 4032,
  'ExifOffset': 238,
  'ExifVersion': b'0220',
  'ExposureBiasValue': (0, 100),
  'ExposureMode': 0,
  'ExposureProgram': 2,
  'ExposureTime': (1, 15),
  'FNumber': (210, 100),
  'Flash': 0,
  'FocalLength': (600, 100),
  'FocalLengthIn35mmFilm': 52,
  'GPSInfo': {1: 'N',
              2: ((37, 1), (30, 1), (30580919, 1000000)),
              3: 'E',
              4: ((127, 1), (6, 1), (12589559, 1000000))},
  'ISOSpeedRatings': 400,
  'ImageLength': 1908,
  'ImageUniqueID': 'O13XLMA00SM',
  'ImageWidth': 4032,
  'Make': 'samsung',
  'MaxApertureValue': (214, 100),
  'MeteringMode': 2,
  'Model': 'SM-N976N',
  'Orientation': 1,
  'ResolutionUnit': 2,
  'SceneCaptureType': 0,
  'ShutterSpeedValue': (1, 15),
 

In [96]:
a = ['640x640 Done. (0.009s)']

640x640 Done. (0.009s)
